In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [4]:
# read file of england_permier_match
folder_path = '/Users/steven/Desktop/Football_Prediction/data/england_premier_match'
# 读取文件夹下的所有CSV文件
file_list = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

data_match_frames = []

for file in file_list:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    # print(len(df))
    data_match_frames.append(df)

match_data = pd.concat(data_match_frames, ignore_index=True)

print(len(match_data))

# read file of england_permier_player
folder_path = '/Users/steven/Desktop/Football_Prediction/data/england_premier_player'
# 读取文件夹下的所有CSV文件
file_list = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

data_player_frames = []

for file in file_list:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    data_player_frames.append(df)

player_data = pd.concat(data_player_frames, ignore_index=True)

print(len(player_data))


# read file of england_permier_team
folder_path = '/Users/steven/Desktop/Football_Prediction/data/england_premier_team'
# 读取文件夹下的所有CSV文件
file_list = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

data_team_frames = []

for file in file_list:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    data_team_frames.append(df)

team_data = pd.concat(data_team_frames, ignore_index=True)

print(len(team_data))

4940
8076
260


In [5]:
# 计算胜率 new feature
team_data_grouped = team_data.groupby('team_name').agg({'wins': 'sum', 'matches_played': 'sum'}).reset_index()


team_data_grouped['win_rate'] = team_data_grouped['wins'] / team_data_grouped['matches_played']
team_data_sorted = team_data_grouped.sort_values(by='win_rate', ascending=False)
print(team_data_sorted[['team_name', 'win_rate']])

                     team_name  win_rate
20          Manchester City FC  0.686235
19                Liverpool FC  0.564777
21        Manchester United FC  0.560729
1                   Arsenal FC  0.546559
11                  Chelsea FC  0.540486
33        Tottenham Hotspur FC  0.530364
18           Leicester City FC  0.391813
7                 Brentford FC  0.368421
13                  Everton FC  0.368421
23         Newcastle United FC  0.337719
36          West Ham United FC  0.331140
38  Wolverhampton Wanderers FC  0.319549
12           Crystal Palace FC  0.313158
29              Southampton FC  0.311005
32            Swansea City AFC  0.308271
30               Stoke City FC  0.305921
17             Leeds United FC  0.298246
0              AFC Bournemouth  0.293860
8    Brighton & Hove Albion FC  0.289474
6          Bolton Wanderers FC  0.289474
2               Aston Villa FC  0.284211
9                   Burnley FC  0.281955
28         Sheffield United FC  0.276316
34              

In [6]:
team_data = team_data.merge(team_data_grouped[['team_name', 'win_rate']], on='team_name', how='left')
print(team_data)

                      team_name              common_name     season  country  \
0                    Arsenal FC                  Arsenal  2013/2014  England   
1          Tottenham Hotspur FC        Tottenham Hotspur  2013/2014  England   
2            Manchester City FC          Manchester City  2013/2014  England   
3                 Stoke City FC               Stoke City  2013/2014  England   
4       West Bromwich Albion FC     West Bromwich Albion  2013/2014  England   
..                          ...                      ...        ...      ...   
255              Aston Villa FC              Aston Villa  2019/2020  England   
256             Norwich City FC             Norwich City  2019/2020  England   
257   Brighton & Hove Albion FC   Brighton & Hove Albion  2019/2020  England   
258  Wolverhampton Wanderers FC  Wolverhampton Wanderers  2019/2020  England   
259         Sheffield United FC         Sheffield United  2019/2020  England   

     matches_played  matches_played_hom

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from math import sqrt

In [8]:
# delete 非数值的feature
feature_cols = ['matches_played', 'matches_played_home', 
                'matches_played_away', 'suspended_matches', 'wins', 'wins_home', 'wins_away', 'draws', 'draws_home', 'draws_away',
                  'losses', 'losses_home', 'losses_away', 'points_per_game', 'points_per_game_home', 'points_per_game_away', 'league_position', 
                  'league_position_home', 'league_position_away', 'performance_rank', 'goals_scored', 'goals_conceded', 'goal_difference', 'total_goal_count',
                    'total_goal_count_home', 'total_goal_count_away', 'goals_scored_home', 'goals_scored_away', 'goals_conceded_home', 'goals_conceded_away', 
                    'goal_difference_home', 'goal_difference_away', 'minutes_per_goal_scored', 'minutes_per_goal_scored_home', 'minutes_per_goal_scored_away', 
                    'minutes_per_goal_conceded', 'minutes_per_goal_conceded_home', 'minutes_per_goal_conceded_away', 'clean_sheets', 'clean_sheets_home', 
                    'clean_sheets_away', 'btts_count', 'btts_count_home', 'btts_count_away', 'fts_count', 'fts_count_home', 'fts_count_away',
                    'first_team_to_score_count', 'first_team_to_score_count_home', 'first_team_to_score_count_away', 'corners_total', 'corners_total_home', 
                    'corners_total_away', 'cards_total', 'cards_total_home', 'cards_total_away', 'average_possession', 'average_possession_home', 
                    'average_possession_away', 'shots', 'shots_home', 'shots_away', 'shots_on_target', 'shots_on_target_home', 'shots_on_target_away', 
                    'shots_off_target', 'shots_off_target_home', 'shots_off_target_away', 'fouls', 'fouls_home', 'fouls_away', 'goals_scored_half_time', 
                    'goals_scored_half_time_home', 'goals_scored_half_time_away', 'goals_conceded_half_time', 'goals_conceded_half_time_home',
                      'goals_conceded_half_time_away', 'goal_difference_half_time', 'goal_difference_half_time_home', 'goal_difference_half_time_away', 
                      'leading_at_half_time', 'leading_at_half_time_home', 'leading_at_half_time_away', 'draw_at_half_time', 'draw_at_half_time_home',
                        'draw_at_half_time_away', 'losing_at_half_time', 'losing_at_half_time_home', 'losing_at_half_time_away', 'points_per_game_half_time',
                          'points_per_game_half_time_home', 'points_per_game_half_time_away', 'average_total_goals_per_match', 'average_total_goals_per_match_home',
                            'average_total_goals_per_match_away', 'goals_scored_per_match', 'goals_scored_per_match_home', 'goals_scored_per_match_away', 
                            'goals_conceded_per_match', 'goals_conceded_per_match_home', 'goals_conceded_per_match_away', 'total_goals_per_match_half_time', 
                            'total_goals_per_match_half_time_home', 'total_goals_per_match_half_time_away']



# //回归模型

X = team_data[feature_cols]
y = team_data['win_rate']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

linear_regression = LinearRegression()
decision_tree = DecisionTreeRegressor()
random_forest = RandomForestRegressor()

models = [linear_regression, decision_tree, random_forest]
model_names = ['Linear Regression', 'Decision Tree', 'Random Forest']
for model, name in zip(models, model_names):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    rmse = sqrt(mean_squared_error(y_test, predictions))
    print(f'{name} RMSE: {rmse}')






Linear Regression RMSE: 0.0727108819823603
Decision Tree RMSE: 0.1152602704463895
Random Forest RMSE: 0.06962448118996971
